In [1]:

import importlib

#import autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
import pwd
import numpy as np
from pathlib import Path
from tqdm import tqdm, trange
from tqdm.contrib.concurrent import process_map
sys.path.append('../')
from meth import tools as mtools
import matplotlib.pyplot as plt
import GEOparse
import scanpy as sc
from scipy import sparse as sp
# %autoreload 2

# GSE 121690

In [8]:
cwd = Path.cwd()
# cwd = Path('/Users/tarkhov/Dropbox/Harvard/Science/Church/')
project_dir = Path.joinpath(cwd, Path('data/GSE121690/'))
project_id = 'GSE121690'
project_samples_dir = 'GSE121690_RAW'
metainfo_path = 'GSE121690_series_matrix.txt'
gl_metainfo_path = Path.joinpath(project_dir, metainfo_path)

In [9]:
import GEOparse
gse = GEOparse.get_GEO(geo="GSE121690", destdir="./data/GSE121690/")

26-Sep-2022 14:23:02 DEBUG utils - Directory ./data/GSE121690/ already exists. Skipping.
26-Sep-2022 14:23:02 INFO GEOparse - File already exist: using local version.
26-Sep-2022 14:23:02 INFO GEOparse - Parsing ./data/GSE121690/GSE121690_family.soft.gz: 
26-Sep-2022 14:23:02 DEBUG GEOparse - DATABASE: GeoMiame
26-Sep-2022 14:23:02 DEBUG GEOparse - SERIES: GSE121690
26-Sep-2022 14:23:02 DEBUG GEOparse - PLATFORM: GPL19057
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443369
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443370
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443371
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443372
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443373
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443374
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443375
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443376
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443377
26-Sep-2022 14:23:03 DEBUG GEOparse - SAMPLE: GSM3443378
26-S

In [10]:
df_meta = gse.phenotype_data.copy()
df_meta.shape
df_meta.head()

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,...,contact_country,instrument_model,library_selection,library_source,library_strategy,relation,supplementary_file_1,supplementary_file_2,series_id,data_row_count
GSM3443369,E4.5-5.5_new_Plate1_A02 (scBS-Seq),GSM3443369,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,United Kingdom,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0
GSM3443370,E4.5-5.5_new_Plate1_A03 (scBS-Seq),GSM3443370,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,United Kingdom,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0
GSM3443371,E4.5-5.5_new_Plate1_A04 (scBS-Seq),GSM3443371,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,United Kingdom,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0
GSM3443372,E4.5-5.5_new_Plate1_A05 (scBS-Seq),GSM3443372,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,United Kingdom,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0
GSM3443373,E4.5-5.5_new_Plate1_A07 (scBS-Seq),GSM3443373,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,United Kingdom,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0


In [11]:
df_meta = gse.phenotype_data.copy()
df_meta_2 = df_meta.copy()
df_meta['Name PerCov'] = df_meta['geo_accession'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta['Data Type'] = 'percentage'
df_meta_2['Name PerCov'] = df_meta_2['geo_accession'].str.cat(['coverage'] * df_meta.shape[0], sep='_')
df_meta_2['Data Type'] = 'coverage'
df_meta['Name PerCov'] = df_meta['geo_accession'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta = pd.concat((df_meta, df_meta_2), axis=0)
df_meta.set_index('Name PerCov', inplace=True)
print(df_meta.shape)
df_meta.head()

(1516, 35)


,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,...,instrument_model,library_selection,library_source,library_strategy,relation,supplementary_file_1,supplementary_file_2,series_id,data_row_count,Data Type
Name PerCov,,,,,,,,,,,,,,,,,,,,,
GSM3443369_percentage,E4.5-5.5_new_Plate1_A02 (scBS-Seq),GSM3443369,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage
GSM3443370_percentage,E4.5-5.5_new_Plate1_A03 (scBS-Seq),GSM3443370,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage
GSM3443371_percentage,E4.5-5.5_new_Plate1_A04 (scBS-Seq),GSM3443371,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage
GSM3443372_percentage,E4.5-5.5_new_Plate1_A05 (scBS-Seq),GSM3443372,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage
GSM3443373_percentage,E4.5-5.5_new_Plate1_A07 (scBS-Seq),GSM3443373,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,10090,...,Illumina NextSeq 500,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage


In [30]:
df_meta['characteristics_ch1.1.developmental stage'].value_counts()
df_meta['Age'] = df_meta['characteristics_ch1.1.developmental stage'].apply(lambda x: float(x[1:]) / 365. * 12)
df_meta['Mouse_ID'] = df_meta['characteristics_ch1.1.developmental stage']
mice = df_meta['Mouse_ID'].unique()
mice = np.array(['E4.5', 'E5.5', 'E6.5', 'E7.5'])

In [31]:
data_path = Path.joinpath(cwd, project_dir, "{}_coverage_0_binned_by_1.h5ad".format(project_id))
print(data_path)
ad = sc.read_h5ad(data_path)
ad.var = pd.concat((ad.var, df_meta.loc[ad.var.index,:]), axis=1)
ad.X = ad.X.astype(np.float32)

/Users/tarkhov/Dropbox/Harvard/Science/scDNAm/data/GSE121690/GSE121690_coverage_0_binned_by_1.h5ad


In [32]:
ad.obs_names_make_unique()
ad.var_names_make_unique()

In [33]:
ad.obs_names_make_unique()
ad.var_names_make_unique()
ad.write_h5ad(Path.joinpath(project_dir, "{}_wmeta.h5ad".format(project_id)))

/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'title' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'geo_accession' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated 

In [34]:
df_meta.iloc[:,6:].head()

,channel_count,source_name_ch1,organism_ch1,taxid_ch1,characteristics_ch1.0.strain,characteristics_ch1.1.developmental stage,characteristics_ch1.2.sample type,molecule_ch1,extract_protocol_ch1,data_processing,...,library_source,library_strategy,relation,supplementary_file_1,supplementary_file_2,series_id,data_row_count,Data Type,Age,Mouse_ID
Name PerCov,,,,,,,,,,,,,,,,,,,,,
GSM3443369_percentage,1,Early embryo,Mus musculus,10090,C57BL/6Babr,E5.5,single cell,genomic DNA,Single-cells were sorted either manually (E4.5...,Libraries were sequenced on the Illumina Nexts...,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.180822,E5.5
GSM3443370_percentage,1,Early embryo,Mus musculus,10090,C57BL/6Babr,E5.5,single cell,genomic DNA,Single-cells were sorted either manually (E4.5...,Libraries were sequenced on the Illumina Nexts...,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.180822,E5.5
GSM3443371_percentage,1,Early embryo,Mus musculus,10090,C57BL/6Babr,E5.5,single cell,genomic DNA,Single-cells were sorted either manually (E4.5...,Libraries were sequenced on the Illumina Nexts...,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.180822,E5.5
GSM3443372_percentage,1,Early embryo,Mus musculus,10090,C57BL/6Babr,E5.5,single cell,genomic DNA,Single-cells were sorted either manually (E4.5...,Libraries were sequenced on the Illumina Nexts...,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.180822,E5.5
GSM3443373_percentage,1,Early embryo,Mus musculus,10090,C57BL/6Babr,E5.5,single cell,genomic DNA,Single-cells were sorted either manually (E4.5...,Libraries were sequenced on the Illumina Nexts...,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.180822,E5.5


In [35]:
x = ad.X[:,::2]
xcov = ad.X[:,1::2]
xmeth = x.multiply(xcov)

mouse_ids = ad.var.loc[::2,:].Mouse_ID.values
mice = np.unique(mouse_ids)
print(mice)
xbulk = np.zeros((ad.X.shape[0], 2* len(mice)))
#xcovbulk = np.zeros((ad.X.shape[0], len(mice)))
for i,mouse_id in enumerate(mice):
    idx = mouse_ids == mouse_id
    xbulk[:,2*i] = xmeth[:,idx].sum(axis=1).flatten() / xcov[:,idx].sum(axis=1).flatten()
    xbulk[:,2*i+1] = xcov[:,idx].sum(axis=1).flatten()
    
mice_index = np.array([[f"{x}_percentage", f"{x}_coverage"] for x in mice]).flatten()

['E4.5' 'E5.5' 'E6.5' 'E7.5']


/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_6997/1600779323.py:12: RuntimeWarning: invalid value encountered in true_divide
  xbulk[:,2*i] = xmeth[:,idx].sum(axis=1).flatten() / xcov[:,idx].sum(axis=1).flatten()


In [38]:
df_meta_bulk = df_meta.drop_duplicates('Mouse_ID').set_index('Mouse_ID').loc[mice].copy()
df_meta_bulk.reset_index(inplace=True)
df_meta_2 = df_meta_bulk.copy()
df_meta_bulk['Name PerCov'] = df_meta_bulk['Mouse_ID'].str.cat(['percentage'] * df_meta_bulk.shape[0], sep='_')
df_meta_bulk['Data Type'] = 'percentage'
df_meta_2['Name PerCov'] = df_meta_2['Mouse_ID'].str.cat(['coverage'] * df_meta_bulk.shape[0], sep='_')
df_meta_2['Data Type'] = 'coverage'
#df_meta['Name PerCov'] = df_meta['Sample Name'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta_bulk = pd.concat((df_meta_bulk, df_meta_2), axis=0)
df_meta_bulk.set_index('Name PerCov', inplace=True)
print(df_meta_bulk.shape)
df_meta_bulk = df_meta_bulk.loc[mice_index]
df_meta_bulk

(8, 37)


,Mouse_ID,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,...,library_selection,library_source,library_strategy,relation,supplementary_file_1,supplementary_file_2,series_id,data_row_count,Data Type,Age
Name PerCov,,,,,,,,,,,,,,,,,,,,,
E4.5_percentage,E4.5,E4.5-5.5_new_Plate3_A02 (scBS-Seq),GSM3443503,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,...,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.147945
E4.5_coverage,E4.5,E4.5-5.5_new_Plate3_A02 (scBS-Seq),GSM3443503,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,...,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,coverage,0.147945
E5.5_percentage,E5.5,E4.5-5.5_new_Plate1_A02 (scBS-Seq),GSM3443369,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,...,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.180822
E5.5_coverage,E5.5,E4.5-5.5_new_Plate1_A02 (scBS-Seq),GSM3443369,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,...,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,coverage,0.180822
E6.5_percentage,E6.5,E6.5_Plate2_G6 (scBS-Seq),GSM3443636,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,...,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.213699
E6.5_coverage,E6.5,E6.5_Plate2_G6 (scBS-Seq),GSM3443636,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,...,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,coverage,0.213699
E7.5_percentage,E7.5,E7.5_Plate1_A3 (scBS-Seq),GSM3443831,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,...,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,percentage,0.246575
E7.5_coverage,E7.5,E7.5_Plate1_A3 (scBS-Seq),GSM3443831,Public on Sep 30 2019,Oct 23 2018,Sep 30 2019,SRA,1,Early embryo,Mus musculus,...,RANDOM,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3443...,"GSE121690,GSE121708",0,coverage,0.246575


In [39]:
adbu = sc.AnnData(sp.csr_matrix(np.nan_to_num(xbulk)), ad.obs, df_meta_bulk)
adbu.obs_names_make_unique()
adbu.var_names_make_unique()

adbu.write_h5ad(Path.joinpath(project_dir, "{}_bulked_wmeta_mm10.h5ad".format(project_id)))

/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Mouse_ID' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'title' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and w

In [41]:
adbu, ad

(AnnData object with n_obs × n_vars = 20073742 × 8
     var: 'Mouse_ID', 'title', 'geo_accession', 'status', 'submission_date', 'last_update_date', 'type', 'channel_count', 'source_name_ch1', 'organism_ch1', 'taxid_ch1', 'characteristics_ch1.0.strain', 'characteristics_ch1.1.developmental stage', 'characteristics_ch1.2.sample type', 'molecule_ch1', 'extract_protocol_ch1', 'data_processing', 'platform_id', 'contact_name', 'contact_email', 'contact_department', 'contact_institute', 'contact_address', 'contact_city', 'contact_zip/postal_code', 'contact_country', 'instrument_model', 'library_selection', 'library_source', 'library_strategy', 'relation', 'supplementary_file_1', 'supplementary_file_2', 'series_id', 'data_row_count', 'Data Type', 'Age',
 AnnData object with n_obs × n_vars = 20073742 × 1516
     var: 'title', 'geo_accession', 'status', 'submission_date', 'last_update_date', 'type', 'channel_count', 'source_name_ch1', 'organism_ch1', 'taxid_ch1', 'characteristics_ch1.0.strain', 

In [48]:
ad[:, ad.var.Age * 365 / 12 > 5.].write_h5ad(Path.joinpath(project_dir, "{}_wmeta_postE55.h5ad".format(project_id)))

In [49]:
adbu[:, adbu.var.Age * 365 / 12 > 5.].write_h5ad(Path.joinpath(project_dir, "{}_bulked_wmeta_postE55.h5ad".format(project_id)))

In [46]:
ad.var.Age * 365 / 12

GSM3443850_percentage    7.5
GSM3443850_coverage      7.5
GSM3443761_percentage    6.5
GSM3443761_coverage      6.5
GSM3443588_percentage    4.5
                        ... 
GSM3443956_coverage      7.5
GSM3443841_percentage    7.5
GSM3443841_coverage      7.5
GSM3443717_percentage    6.5
GSM3443717_coverage      6.5
Name: Age, Length: 1516, dtype: float64

# GSE56879

In [71]:
gse = 'GSE56879'
cwd = Path.cwd()
# cwd = Path('/Users/tarkhov/Dropbox/Harvard/Science/Church/')
project_dir = Path.joinpath(cwd, Path(f'data/{gse}/'))
project_id = f'{gse}'
project_samples_dir = f'{gse}_RAW'
metainfo_path = f'{gse}_series_matrix.txt'
gl_metainfo_path = Path.joinpath(project_dir, metainfo_path)

In [72]:
import GEOparse
gse = GEOparse.get_GEO(geo=f"{gse}", destdir=f"./data/{gse}/")

03-Aug-2022 20:54:02 DEBUG utils - Directory ./data/GSE56879/ already exists. Skipping.
03-Aug-2022 20:54:02 INFO GEOparse - File already exist: using local version.
03-Aug-2022 20:54:02 INFO GEOparse - Parsing ./data/GSE56879/GSE56879_family.soft.gz: 
03-Aug-2022 20:54:02 DEBUG GEOparse - DATABASE: GeoMiame
03-Aug-2022 20:54:02 DEBUG GEOparse - SERIES: GSE56879
03-Aug-2022 20:54:02 DEBUG GEOparse - PLATFORM: GPL13112
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370522
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370523
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370524
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370525
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370526
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370527
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370528
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370529
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370530
03-Aug-2022 20:54:02 DEBUG GEOparse - SAMPLE: GSM1370531
03-Aug-2

In [73]:
df_meta = gse.phenotype_data.copy()
df_meta_2 = df_meta.copy()
df_meta['Name PerCov'] = df_meta['geo_accession'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta['Data Type'] = 'percentage'
df_meta_2['Name PerCov'] = df_meta_2['geo_accession'].str.cat(['coverage'] * df_meta.shape[0], sep='_')
df_meta_2['Data Type'] = 'coverage'
df_meta['Name PerCov'] = df_meta['geo_accession'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta = pd.concat((df_meta, df_meta_2), axis=0)
df_meta.set_index('Name PerCov', inplace=True)
print(df_meta.shape)
df_meta.head()

(112, 39)


,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,...,relation,supplementary_file_1,supplementary_file_2,series_id,data_row_count,characteristics_ch1.1.treatment,characteristics_ch1.2.strain,characteristics_ch1.0.strain,description,Data Type
Name PerCov,,,,,,,,,,,,,,,,,,,,,
GSM1370522_percentage,MII_1,GSM1370522,Public on Jul 18 2014,Apr 17 2014,May 15 2019,SRA,1,oocytes,Mus musculus,10090,...,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,GSE56879,0,NaN,NaN,NaN,NaN,percentage
GSM1370523_percentage,MII_2,GSM1370523,Public on Jul 18 2014,Apr 17 2014,May 15 2019,SRA,1,oocytes,Mus musculus,10090,...,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,GSE56879,0,NaN,NaN,NaN,NaN,percentage
GSM1370524_percentage,MII_3,GSM1370524,Public on Jul 18 2014,Apr 17 2014,May 15 2019,SRA,1,oocytes,Mus musculus,10090,...,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,GSE56879,0,NaN,NaN,NaN,NaN,percentage
GSM1370525_percentage,MII_4,GSM1370525,Public on Jul 18 2014,Apr 17 2014,May 15 2019,SRA,1,oocytes,Mus musculus,10090,...,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,GSE56879,0,NaN,NaN,NaN,NaN,percentage
GSM1370526_percentage,MII_5,GSM1370526,Public on Jul 18 2014,Apr 17 2014,May 15 2019,SRA,1,oocytes,Mus musculus,10090,...,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1370...,GSE56879,0,NaN,NaN,NaN,NaN,percentage


In [222]:
df_meta.columns

Index(['Run', 'Assay Type', 'AvgSpotLen', 'Bases', 'BioProject', 'BioSample',
       'Bytes', 'Center Name', 'Consent', 'DATASTORE filetype',
       'DATASTORE provider', 'DATASTORE region', 'Experiment',
       'GEO_Accession (exp)', 'Instrument', 'LibraryLayout',
       'LibrarySelection', 'LibrarySource', 'Organism', 'Platform_x',
       'ReleaseDate', 'Sample Name', 'SRA Study', 'Age', 'Cell_type',
       'Mouse_ID', 'source_name', 'STRAIN', 'Accession', 'Title',
       'Sample Type', 'Taxonomy', 'Channels', 'Platform_y', 'Series',
       'Supplementary Types', 'Supplementary Links', 'SRA Accession',
       'Contact', 'Release Date'],
      dtype='object')

In [19]:
data_path = Path.joinpath(cwd, project_dir, "{}_coverage_0_binned_by_1.h5ad".format(project_id))
print(data_path)
ad = sc.read_h5ad(data_path)
ad.var = pd.concat((ad.var, df_meta.loc[ad.var.index,:]), axis=1)
ad.X = ad.X.astype(np.float32)

/Users/tarkhov/Dropbox/Harvard/Science/scDNAm/data/GSE56879/GSE56879_coverage_0_binned_by_1.h5ad


In [20]:
ad.obs_names_make_unique()
ad.var_names_make_unique()

In [21]:
ad.write_h5ad(Path.joinpath(project_dir, "{}_wmeta.h5ad".format(project_id)))

/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'title' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'geo_accession' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated 

In [28]:
df_meta_MII = df_meta.copy()

In [5]:
df = pd.read_csv(path_sample, sep='\t', header=None, skiprows=1,
                 dtype={0: 'str', 1: 'int', 2: 'int', 3: 'float', 4: 'int', 5: 'int'})
df.head()

,0,1,2,3,4,5
0,8,3021291,3021291,100.0,1,0
1,8,3021293,3021293,100.0,1,0
2,8,3021452,3021452,0.0,0,1
3,8,3036686,3036686,100.0,1,0
4,8,3068337,3068337,100.0,1,0


,0,1,2,3,4,5
0,8,3021291,3021291,100.0,1,0
1,8,3021293,3021293,100.0,1,0
2,8,3021452,3021452,0.0,0,1
3,8,3036686,3036686,100.0,1,0
4,8,3068337,3068337,100.0,1,0
...,...,...,...,...,...,...
7644136,Y,90829081,90829081,0.0,0,1
7644137,Y,90829087,90829087,0.0,0,1
7644138,Y,90829099,90829099,0.0,0,1
7644139,Y,90829119,90829119,0.0,0,1


# GSE186458

In [3]:
cwd = Path.cwd()
# cwd = Path('/Users/tarkhov/Dropbox/Harvard/Science/Church/')
project_dir = Path.joinpath(cwd, Path('data/GSE186458/'))
project_id = 'GSE186458'
project_samples_dir = 'GSE186458_RAW'
metainfo_path = 'GSE186458_series_matrix.txt'
gl_metainfo_path = Path.joinpath(project_dir, metainfo_path)

In [22]:
import GEOparse
gse = GEOparse.get_GEO(geo="GSE186458", destdir="./data/GSE186458/")

16-Jun-2022 12:44:03 DEBUG utils - Directory ./data/GSE186458/ already exists. Skipping.
16-Jun-2022 12:44:03 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE186nnn/GSE186458/soft/GSE186458_family.soft.gz to ./data/GSE186458/GSE186458_family.soft.gz
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22.3k/22.3k [00:00<00:00, 289kB/s]
16-Jun-2022 12:44:03 DEBUG downloader - Size validation passed
16-Jun-2022 12:44:03 DEBUG downloader - Moving /var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/tmppv7byq60 to /Users/tarkhov/Dropbox/Harvard/Science/scDNAm/data/GSE186458/GSE186458_family.soft.gz
16-Jun-2022 12:44:03 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE186nnn/GSE186458/soft/GSE186458_family.soft.gz
16-Jun-2022 12:44:03 INFO GEOparse - Parsing ./data/GSE186458/GSE186458_family.soft.gz: 
16-Jun-2022 12:44:03 DEBUG GEOparse - DATABASE: GeoMiame
16-Jun-2022 12:44:03 

In [40]:
df_meta = gse.phenotype_data.copy()
df_meta.head()

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,...,library_source,library_strategy,relation,supplementary_file_1,supplementary_file_2,supplementary_file_3,supplementary_file_4,series_id,data_row_count,description
GSM5652176,Adipocytes-Z000000T7,GSM5652176,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Abdominal Subcut.,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN
GSM5652177,Adipocytes-Z000000T9,GSM5652177,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Abdominal Subcut.,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN
GSM5652178,Adipocytes-Z000000T5,GSM5652178,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Abdominal Subcut.,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN
GSM5652179,Aorta-Endothelium-Z00000422,GSM5652179,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Aorta,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN
GSM5652180,Aorta-Endothelium-Z0000043G,GSM5652180,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Aorta,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN


,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,taxid_ch1,...,library_source,library_strategy,relation,supplementary_file_1,supplementary_file_2,supplementary_file_3,supplementary_file_4,series_id,data_row_count,description
GSM5652176,Adipocytes-Z000000T7,GSM5652176,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Abdominal Subcut.,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN
GSM5652177,Adipocytes-Z000000T9,GSM5652177,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Abdominal Subcut.,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN
GSM5652178,Adipocytes-Z000000T5,GSM5652178,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Abdominal Subcut.,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN
GSM5652179,Aorta-Endothelium-Z00000422,GSM5652179,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Aorta,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN
GSM5652180,Aorta-Endothelium-Z0000043G,GSM5652180,Public on Mar 02 2022,Oct 23 2021,May 18 2022,SRA,1,Aorta,Homo sapiens,9606,...,genomic,Bisulfite-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5652...,GSE186458,0,NaN


# SRA344045 liver

In [70]:
cwd = Path.cwd()
# cwd = Path('/Users/tarkhov/Dropbox/Harvard/Science/Church/')
project_dir = Path.joinpath(cwd, Path('data/SRA344045/'))
project_id = 'SRA344045'
project_samples_dir = 'SRA344045_RAW'
metainfo_path = 'SraRunTable.txt'
gl_metainfo_path = Path.joinpath(project_dir, metainfo_path)

In [71]:
df_meta = pd.read_csv(gl_metainfo_path, sep=',')
df_meta['Age_months'] = df_meta['Age'].copy()
df_meta['Age_months'].fillna('0 months', inplace=True)
df_meta['Age'] = df_meta['Age_months'].apply(lambda x: float(x.split(' ')[0]))
df_meta['Mouse_ID'] = df_meta['Sample Name'].apply(lambda x: x[:2])
print(df_meta.shape)
df_meta_2 = df_meta.copy()
df_meta['Name PerCov'] = df_meta['Run'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta['Data Type'] = 'percentage'
df_meta_2['Name PerCov'] = df_meta_2['Run'].str.cat(['coverage'] * df_meta.shape[0], sep='_')
df_meta_2['Data Type'] = 'coverage'
df_meta['Name PerCov'] = df_meta['Run'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta = pd.concat((df_meta, df_meta_2), axis=0)
df_meta.set_index('Name PerCov', inplace=True)
print(df_meta.shape)
df_meta.head()

(34, 32)
(68, 33)


,Run,Assay Type,AssemblyName,AvgSpotLen,Bases,BioProject,BioSample,BioSampleModel,Bytes,Center Name,...,Sample Name,sex,SRA Study,strain,tissue,Age,dev_stage,Age_months,Mouse_ID,Data Type
Name PerCov,,,,,,,,,,,,,,,,,,,,,
SRR3136624_percentage,SRR3136624,Bisulfite-Seq,GCA_000001635.1,42,476309309,PRJNA310298,SAMN04449570,Model organism or animal,305774648,ALBERT EINSTEIN COLLEGE OF MEDICINE,...,f11,male,SRP069120,C57BL/6,MEFs in culture,0.0,embryonic,0 months,f1,percentage
SRR3136625_percentage,SRR3136625,Bisulfite-Seq,GCA_000001635.1,82,276514225,PRJNA310298,SAMN04449998,Model organism or animal,166102905,ALBERT EINSTEIN COLLEGE OF MEDICINE,...,f23,male,SRP069120,C57BL/6,MEFs in culture,0.0,embryonic,0 months,f2,percentage
SRR3136626_percentage,SRR3136626,Bisulfite-Seq,GCA_000001635.1,83,345410020,PRJNA310298,SAMN04449847,Model organism or animal,197726064,ALBERT EINSTEIN COLLEGE OF MEDICINE,...,f22,male,SRP069120,C57BL/6,MEFs in culture,0.0,embryonic,0 months,f2,percentage
SRR3136627_percentage,SRR3136627,Bisulfite-Seq,GCA_000001635.1,85,450783127,PRJNA310298,SAMN04449846,Model organism or animal,250903561,ALBERT EINSTEIN COLLEGE OF MEDICINE,...,f21,male,SRP069120,C57BL/6,MEFs in culture,0.0,embryonic,0 months,f2,percentage
SRR3136628_percentage,SRR3136628,Bisulfite-Seq,GCA_000001635.1,83,322920190,PRJNA310298,SAMN04449988,Model organism or animal,214862269,ALBERT EINSTEIN COLLEGE OF MEDICINE,...,o11,male,SRP069120,C57BL/6,Liver,26.0,NaN,26 month,o1,percentage


In [72]:
df_meta.info()
to_category = np.setdiff1d(df_meta.columns, ['Age'])
df_meta[to_category] = df_meta[to_category].astype("category")
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, SRR3136624_percentage to SRR3136673_coverage
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Run                 68 non-null     object 
 1   Assay Type          68 non-null     object 
 2   AssemblyName        68 non-null     object 
 3   AvgSpotLen          68 non-null     int64  
 4   Bases               68 non-null     int64  
 5   BioProject          68 non-null     object 
 6   BioSample           68 non-null     object 
 7   BioSampleModel      68 non-null     object 
 8   Bytes               68 non-null     int64  
 9   Center Name         68 non-null     object 
 10  Consent             68 non-null     object 
 11  DATASTORE filetype  68 non-null     object 
 12  DATASTORE provider  68 non-null     object 
 13  DATASTORE region    68 non-null     object 
 14  Experiment          68 non-null     object 
 15  Instrument          68 non-

In [73]:
data_path = Path.joinpath(cwd, project_dir, "{}_coverage_0_binned_by_1.h5ad".format(project_id))
print(data_path)
ad = sc.read_h5ad(data_path)
ad.var = pd.concat((ad.var, df_meta.loc[ad.var.index,:]), axis=1)
ad.X = ad.X.astype(np.float32)

/Users/tarkhov/Dropbox/Harvard/Science/scDNAm/data/SRA344045/SRA344045_coverage_0_binned_by_1.h5ad


In [74]:
# ad.X

In [75]:
ad[:,ad.var.tissue.values == 'Liver'].write_h5ad(Path.joinpath(project_dir, "{}_Liver_wmeta.h5ad".format(project_id)))

In [76]:
ad[:,ad.var.tissue.values != 'Liver'].write_h5ad(Path.joinpath(project_dir, "{}_MEF_wmeta.h5ad".format(project_id)))

In [77]:
ad.write_h5ad(Path.joinpath(project_dir, "{}_wmeta.h5ad".format(project_id)))

In [78]:
# mouse_ids = ad.var.loc[::2,:].Mouse_ID.values
ad.var['Mouse_ID'] = ad.var['Sample Name'].apply(lambda x: x[:2])
# ad.var.loc[::2,:].columns
# ad.var.iloc[:,5:].head(3)
ad.var['Mouse_ID'].sort_values()#value_counts()#iloc[:,5:].head(3)

SRR3136634_percentage    f1
SRR3136634_coverage      f1
SRR3136624_percentage    f1
SRR3136624_coverage      f1
SRR3136627_coverage      f2
SRR3136627_percentage    f2
SRR3136626_coverage      f2
SRR3136626_percentage    f2
SRR3136625_percentage    f2
SRR3136625_coverage      f2
SRR3136630_percentage    o1
SRR3136653_coverage      o1
SRR3136653_percentage    o1
SRR3136630_coverage      o1
SRR3136629_percentage    o1
SRR3136629_coverage      o1
SRR3136628_coverage      o1
SRR3136646_percentage    o1
SRR3136651_percentage    o1
SRR3136651_coverage      o1
SRR3136646_coverage      o1
SRR3136628_percentage    o1
SRR3136631_coverage      o2
SRR3136631_percentage    o2
SRR3136635_coverage      o2
SRR3136635_percentage    o2
SRR3136647_coverage      o3
SRR3136652_percentage    o3
SRR3136652_coverage      o3
SRR3136647_percentage    o3
SRR3136658_coverage      y1
SRR3136658_percentage    y1
SRR3136655_percentage    y1
SRR3136660_coverage      y1
SRR3136660_percentage    y1
SRR3136655_coverage 

In [79]:
x = ad.X[:,::2]
xcov = ad.X[:,1::2]
xmeth = x.multiply(xcov)

mouse_ids = ad.var.loc[::2,:].Mouse_ID.values
mice = np.unique(mouse_ids)
print(mice)
xbulk = np.zeros((ad.X.shape[0], 2* len(mice)))
#xcovbulk = np.zeros((ad.X.shape[0], len(mice)))
for i,mouse_id in enumerate(mice):
    idx = mouse_ids == mouse_id
    xbulk[:,2*i] = xmeth[:,idx].sum(axis=1).flatten() / xcov[:,idx].sum(axis=1).flatten()
    xbulk[:,2*i+1] = xcov[:,idx].sum(axis=1).flatten()

['f1' 'f2' 'o1' 'o2' 'o3' 'y1' 'y2' 'y3']


/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_1713/3837489246.py:12: RuntimeWarning: invalid value encountered in true_divide
  xbulk[:,2*i] = xmeth[:,idx].sum(axis=1).flatten() / xcov[:,idx].sum(axis=1).flatten()


In [80]:
mice_index = np.array([[f"{x}_percentage", f"{x}_coverage"] for x in mice]).flatten()

In [81]:
df_meta_bulk = df_meta.drop_duplicates('Mouse_ID').set_index('Mouse_ID').loc[mice].copy()
df_meta_bulk.reset_index(inplace=True)
df_meta_2 = df_meta_bulk.copy()
df_meta_bulk['Name PerCov'] = df_meta_bulk['Mouse_ID'].str.cat(['percentage'] * df_meta_bulk.shape[0], sep='_')
df_meta_bulk['Data Type'] = 'percentage'
df_meta_2['Name PerCov'] = df_meta_2['Mouse_ID'].str.cat(['coverage'] * df_meta_bulk.shape[0], sep='_')
df_meta_2['Data Type'] = 'coverage'
#df_meta['Name PerCov'] = df_meta['Sample Name'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta_bulk = pd.concat((df_meta_bulk, df_meta_2), axis=0)
df_meta_bulk.set_index('Name PerCov', inplace=True)
print(df_meta_bulk.shape)
df_meta_bulk = df_meta_bulk.loc[mice_index]
df_meta_bulk

(16, 33)


,Mouse_ID,Run,Assay Type,AssemblyName,AvgSpotLen,Bases,BioProject,BioSample,BioSampleModel,Bytes,...,ReleaseDate,Sample Name,sex,SRA Study,strain,tissue,Age,dev_stage,Age_months,Data Type
Name PerCov,,,,,,,,,,,,,,,,,,,,,
f1_percentage,f1,SRR3136624,Bisulfite-Seq,GCA_000001635.1,42,476309309,PRJNA310298,SAMN04449570,Model organism or animal,305774648,...,2016-03-05T00:00:00Z,f11,male,SRP069120,C57BL/6,MEFs in culture,0.0,embryonic,0 months,percentage
f1_coverage,f1,SRR3136624,Bisulfite-Seq,GCA_000001635.1,42,476309309,PRJNA310298,SAMN04449570,Model organism or animal,305774648,...,2016-03-05T00:00:00Z,f11,male,SRP069120,C57BL/6,MEFs in culture,0.0,embryonic,0 months,coverage
f2_percentage,f2,SRR3136625,Bisulfite-Seq,GCA_000001635.1,82,276514225,PRJNA310298,SAMN04449998,Model organism or animal,166102905,...,2016-03-05T00:00:00Z,f23,male,SRP069120,C57BL/6,MEFs in culture,0.0,embryonic,0 months,percentage
f2_coverage,f2,SRR3136625,Bisulfite-Seq,GCA_000001635.1,82,276514225,PRJNA310298,SAMN04449998,Model organism or animal,166102905,...,2016-03-05T00:00:00Z,f23,male,SRP069120,C57BL/6,MEFs in culture,0.0,embryonic,0 months,coverage
o1_percentage,o1,SRR3136628,Bisulfite-Seq,GCA_000001635.1,83,322920190,PRJNA310298,SAMN04449988,Model organism or animal,214862269,...,2016-03-05T00:00:00Z,o11,male,SRP069120,C57BL/6,Liver,26.0,NaN,26 month,percentage
o1_coverage,o1,SRR3136628,Bisulfite-Seq,GCA_000001635.1,83,322920190,PRJNA310298,SAMN04449988,Model organism or animal,214862269,...,2016-03-05T00:00:00Z,o11,male,SRP069120,C57BL/6,Liver,26.0,NaN,26 month,coverage
o2_percentage,o2,SRR3136631,Bisulfite-Seq,GCA_000001635.1,84,204738803,PRJNA310298,SAMN04449994,Model organism or animal,140159990,...,2016-03-05T00:00:00Z,o21,male,SRP069120,C57BL/6,Liver,26.0,NaN,26 month,percentage
o2_coverage,o2,SRR3136631,Bisulfite-Seq,GCA_000001635.1,84,204738803,PRJNA310298,SAMN04449994,Model organism or animal,140159990,...,2016-03-05T00:00:00Z,o21,male,SRP069120,C57BL/6,Liver,26.0,NaN,26 month,coverage
o3_percentage,o3,SRR3136647,Bisulfite-Seq,GCA_000001635.1,83,212610316,PRJNA310298,SAMN04449996,Model organism or animal,127381556,...,2016-03-05T00:00:00Z,o31,male,SRP069120,C57BL/6,Liver,26.0,NaN,26 month,percentage


In [82]:
adbu = sc.AnnData(sp.csr_matrix(np.nan_to_num(xbulk)), ad.obs, df_meta_bulk)
adbu.write_h5ad(Path.joinpath(project_dir, "{}_bulked_wmeta_mm10.h5ad".format(project_id)))

/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Data Type' as categorical


# GSE121364 scRNAseq

In [151]:
cwd = Path.cwd()
# cwd = Path('/Users/tarkhov/Dropbox/Harvard/Science/Church/')
project_dir = Path.joinpath(cwd, Path('data/GSE121364/'))
project_id = 'GSE121364'
project_samples_dir = 'GSE121364_RAW'
metainfo_path = 'SraRunTable.txt'
metainfo_path_2 = 'sample.csv'
gl_metainfo_path = Path.joinpath(project_dir, metainfo_path)
gl_metainfo_path_2 = Path.joinpath(project_dir, metainfo_path_2)
data_path = Path.joinpath(project_dir, "GSE121364_Table_raw_counts_RNA_QSC.txt")

In [152]:
df = pd.read_csv(data_path, sep=' ')
print(df.shape)
df.head(2)

(25952, 380)


/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,geneName,chr,start,lane3_Tom_Y4_TA_Hi_11_CTATCATG_L003_22540_7_R1_val_1_GRCm38_hisat2.bam,lane3_Tom_Y4_TA_Hi_12_TCTCAGTG_L003_22540_7_R1_val_1_GRCm38_hisat2.bam,lane3_Tom_Y4_TA_Hi_13_GCCTGCCC_L003_22540_7_R1_val_1_GRCm38_hisat2.bam,lane3_Tom_Y4_TA_Hi_1_TTCCTGCT_L003_22540_7_R1_val_1_GRCm38_hisat2.bam,lane3_Tom_Y4_TA_Hi_23_CGCCTGAT_L003_22540_7_R1_val_1_GRCm38_hisat2.bam,lane3_Tom_Y4_TA_Hi_25_TTCCATTG_L003_22540_7_R1_val_1_GRCm38_hisat2.bam,lane3_Tom_Y4_TA_Hi_27_CTAGATAG_L003_22540_7_R1_val_1_GRCm38_hisat2.bam,...,lane2_Tom_O5_TA_Hi_84_GTTCCACG_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_85_TTCCTACA_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_86_TCCGGGTA_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_87_GGACACGG_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_89_TACTAGTC_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_90_GATTCATC_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_91_CGACTGCA_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_92_TATTGGAG_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_94_TCGGACCG_L002_22540_5_R1_val_1_GRCm38_hisat2.bam,lane2_Tom_O5_TA_Hi_95_CTTCGGCT_L002_22540_5_R1_val_1_GRCm38_hisat2.bam
0,Xkr4,1,3205901,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Gm1992,1,3466587,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
df_meta = pd.read_csv(gl_metainfo_path, sep=',')
df_meta.head(2)

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,Organism,Platform,ReleaseDate,Sample Name,SRA Study,Age,Cell_type,Mouse_ID,source_name,STRAIN
0,SRR8066856,RNA-Seq,150,1169700,PRJNA497128,SAMN10249431,494375,GEO,public,"fastq,sra",...,Mus musculus,ILLUMINA,2019-08-14T00:00:00Z,GSM3432562,SRP165935,24 months,Muscle stem cells,O1,Tibialis anterior muscles,MGI: 5308730
1,SRR8066857,RNA-Seq,150,147409200,PRJNA497128,SAMN10249430,56823661,GEO,public,"sra,fastq",...,Mus musculus,ILLUMINA,2019-08-14T00:00:00Z,GSM3432563,SRP165935,24 months,Muscle stem cells,O1,Tibialis anterior muscles,MGI: 5308730


In [154]:
df_meta_2 = pd.read_csv(gl_metainfo_path_2, sep=',')
df_meta_2['CellID'] = df_meta_2['Title'].apply(lambda x: x.split(' ')[0])
df_meta_2.head(2)

,Accession,Title,Sample Type,Taxonomy,Channels,Platform,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date,CellID
0,GSM3432562,O1_TA_Hi_10 [scRNA-seq],SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895195,Felix Krueger,"Aug 14, 2019",O1_TA_Hi_10
1,GSM3432563,O1_TA_Hi_11 [scRNA-seq],SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895196,Felix Krueger,"Aug 14, 2019",O1_TA_Hi_11


In [155]:
df_meta_full = pd.merge(df_meta, df_meta_2, left_on='Sample Name', right_on='Accession')
df_meta_full.head(2)

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,Taxonomy,Channels,Platform_y,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date,CellID
0,SRR8066856,RNA-Seq,150,1169700,PRJNA497128,SAMN10249431,494375,GEO,public,"fastq,sra",...,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895195,Felix Krueger,"Aug 14, 2019",O1_TA_Hi_10
1,SRR8066857,RNA-Seq,150,147409200,PRJNA497128,SAMN10249430,56823661,GEO,public,"sra,fastq",...,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895196,Felix Krueger,"Aug 14, 2019",O1_TA_Hi_11


In [156]:
# df_meta_full['CellID'].value_counts()

In [157]:
sampleIDs = df.columns[3:]
print(sampleIDs[:5])
sampleIDs = ['_'.join(x.split('_')[2:6]) for x in sampleIDs]
print(sampleIDs[:5])

Index(['lane3_Tom_Y4_TA_Hi_11_CTATCATG_L003_22540_7_R1_val_1_GRCm38_hisat2.bam',
       'lane3_Tom_Y4_TA_Hi_12_TCTCAGTG_L003_22540_7_R1_val_1_GRCm38_hisat2.bam',
       'lane3_Tom_Y4_TA_Hi_13_GCCTGCCC_L003_22540_7_R1_val_1_GRCm38_hisat2.bam',
       'lane3_Tom_Y4_TA_Hi_1_TTCCTGCT_L003_22540_7_R1_val_1_GRCm38_hisat2.bam',
       'lane3_Tom_Y4_TA_Hi_23_CGCCTGAT_L003_22540_7_R1_val_1_GRCm38_hisat2.bam'],
      dtype='object')
['Y4_TA_Hi_11', 'Y4_TA_Hi_12', 'Y4_TA_Hi_13', 'Y4_TA_Hi_1', 'Y4_TA_Hi_23']


In [158]:
new_cols = df.columns[:3].tolist() + sampleIDs
print(new_cols[:7])
df.columns = new_cols
df.head()

['geneName', 'chr', 'start', 'Y4_TA_Hi_11', 'Y4_TA_Hi_12', 'Y4_TA_Hi_13', 'Y4_TA_Hi_1']


,geneName,chr,start,Y4_TA_Hi_11,Y4_TA_Hi_12,Y4_TA_Hi_13,Y4_TA_Hi_1,Y4_TA_Hi_23,Y4_TA_Hi_25,Y4_TA_Hi_27,...,O5_TA_Hi_84,O5_TA_Hi_85,O5_TA_Hi_86,O5_TA_Hi_87,O5_TA_Hi_89,O5_TA_Hi_90,O5_TA_Hi_91,O5_TA_Hi_92,O5_TA_Hi_94,O5_TA_Hi_95
0,Xkr4,1,3205901,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Gm1992,1,3466587,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rp1,1,4343507,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sox17,1,4490928,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mrpl15,1,4773206,0,0,121,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
df_meta_full.set_index('CellID').loc[sampleIDs].shape, df_meta_full.shape, df.shape

((377, 40), (768, 41), (25952, 380))

In [160]:
df_meta_full['Sample Name'].value_counts()

GSM3432562    1
GSM3432563    1
GSM3433068    1
GSM3433069    1
GSM3433070    1
             ..
GSM3432820    1
GSM3432821    1
GSM3432822    1
GSM3432823    1
GSM3433329    1
Name: Sample Name, Length: 768, dtype: int64

In [161]:
df_meta.iloc[:,10:].head()
df_meta[['Age', 'Mouse_ID']].value_counts()

Age        Mouse_ID
2 months   Y2          96
           Y4          96
           Y5          96
           Y8          96
24 months  O1          96
           O5          96
           O8          96
2 months   Y7          93
dtype: int64

In [162]:
df_full = df.copy().set_index('geneName').T
df_full.head()

geneName,Xkr4,Gm1992,Rp1,Sox17,Mrpl15,Lypla1,Tcea1,Gm6104,Rgs20,Gm16041,...,AC211711.1,AC200894.1,AC192435.1,AC146633.1,AC183902.1,AC151712.3,AC151712.1,AC151712.4,AC151712.5,AC151712.2
chr,1,1,1,1,1,1,1,1,1,1,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
start,3205901,3466587,4343507,4490928,4773206,4807788,4857814,4879202,4909576,4970857,...,88804770,88952381,89328785,89846210,90402688,90754513,90755657,90784738,90838869,90843934
Y4_TA_Hi_11,0,0,0,0,0,0,69,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Y4_TA_Hi_12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Y4_TA_Hi_13,0,0,0,0,121,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
df_meta = df_meta_full.copy()
df_meta.set_index('CellID', inplace=True)
df_meta.head()

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,Sample Type,Taxonomy,Channels,Platform_y,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date
CellID,,,,,,,,,,,,,,,,,,,,,
O1_TA_Hi_10,SRR8066856,RNA-Seq,150,1169700,PRJNA497128,SAMN10249431,494375,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895195,Felix Krueger,"Aug 14, 2019"
O1_TA_Hi_11,SRR8066857,RNA-Seq,150,147409200,PRJNA497128,SAMN10249430,56823661,GEO,public,"sra,fastq",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895196,Felix Krueger,"Aug 14, 2019"
O1_TA_Hi_12,SRR8066858,RNA-Seq,150,185247450,PRJNA497128,SAMN10249429,66440460,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895197,Felix Krueger,"Aug 14, 2019"
O1_TA_Hi_13,SRR8066859,RNA-Seq,150,187086300,PRJNA497128,SAMN10249428,67360076,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895198,Felix Krueger,"Aug 14, 2019"
O1_TA_Hi_14,SRR8066860,RNA-Seq,150,16032900,PRJNA497128,SAMN10249427,5883082,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895199,Felix Krueger,"Aug 14, 2019"


In [193]:
data_path = Path.joinpath(cwd, project_dir, "{}.h5ad".format(project_id))
print(data_path)
ad = sc.AnnData(df_full.iloc[2:,])

/Users/tarkhov/Dropbox/Harvard/Science/scDNAm/data/GSE121364/GSE121364.h5ad


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [194]:
ad.var.head()

""
geneName
Xkr4
Gm1992
Rp1
Sox17
Mrpl15


In [195]:
ad.var['chr'] = df_full.iloc[0,:].values
ad.var['start'] = df_full.iloc[1,:].values
ad.var.head()

,chr,start
geneName,,
Xkr4,1,3205901
Gm1992,1,3466587
Rp1,1,4343507
Sox17,1,4490928
Mrpl15,1,4773206


In [196]:
ad.obs.head()

""
Y4_TA_Hi_11
Y4_TA_Hi_12
Y4_TA_Hi_13
Y4_TA_Hi_1
Y4_TA_Hi_23


In [224]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
z = pca.fit_transform(ad.X)
z.shape

(377, 5)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [231]:
# plt.scatter(z[:,0], z[:,1], c=ad.obs.Age.values == '2 months')
# plt.show()

In [197]:
ad.obs = pd.concat((ad.obs, df_meta.loc[ad.obs.index,:]), axis=1)
ad.obs.head()

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,Sample Type,Taxonomy,Channels,Platform_y,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date
Y4_TA_Hi_11,SRR8067241,RNA-Seq,150,107484600,PRJNA497128,SAMN10250014,41604327,GEO,public,"sra,fastq",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895580,Felix Krueger,"Aug 14, 2019"
Y4_TA_Hi_12,SRR8067242,RNA-Seq,150,90305850,PRJNA497128,SAMN10250013,35342837,GEO,public,"sra,fastq",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895581,Felix Krueger,"Aug 14, 2019"
Y4_TA_Hi_13,SRR8067243,RNA-Seq,150,97244550,PRJNA497128,SAMN10250012,37644383,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895582,Felix Krueger,"Aug 14, 2019"
Y4_TA_Hi_1,SRR8067250,RNA-Seq,150,39392850,PRJNA497128,SAMN10250005,15256244,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895589,Felix Krueger,"Aug 14, 2019"
Y4_TA_Hi_23,SRR8067254,RNA-Seq,150,116119050,PRJNA497128,SAMN10250001,45741310,GEO,public,"sra,fastq",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895593,Felix Krueger,"Aug 14, 2019"


In [198]:
for col in ['AvgSpotLen', 'Bases', 'Bytes', 'Channels']:
    ad.obs[col] = ad.obs[col].astype(str)

for col in ['chr', 'start']:
    ad.var[col] = ad.var[col].astype(str)
    

In [199]:
ad.obs = ad.obs.convert_dtypes()
# ad.obs = ad.obs.drop(['AvgSpotLen', 'Bases', 'Bytes', 'Channels'], axis=1)
# ad.var = ad.var.drop(['chr', 'start'], axis=1)
ad.var = ad.var.convert_dtypes()
ad._sanitize()
ad.obs.info()

/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Assay Type' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'AvgSpotLen' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecate

<class 'pandas.core.frame.DataFrame'>
Index: 377 entries, Y4_TA_Hi_11 to O5_TA_Hi_95
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Run                  377 non-null    string  
 1   Assay Type           377 non-null    category
 2   AvgSpotLen           377 non-null    category
 3   Bases                377 non-null    string  
 4   BioProject           377 non-null    category
 5   BioSample            377 non-null    string  
 6   Bytes                377 non-null    string  
 7   Center Name          377 non-null    category
 8   Consent              377 non-null    category
 9   DATASTORE filetype   377 non-null    category
 10  DATASTORE provider   377 non-null    category
 11  DATASTORE region     377 non-null    category
 12  Experiment           377 non-null    string  
 13  GEO_Accession (exp)  377 non-null    string  
 14  Instrument           377 non-null    category
 15  LibraryLay

In [200]:
ad.obs_names_make_unique()
ad.var_names_make_unique()
ad.write_h5ad(data_path)

In [218]:
ad.to_df().to_csv('../scDNAm/data/GSE121364//For_Orr/X_table.csv')
ad.var.to_csv('../scDNAm/data/GSE121364//For_Orr/Variables_table.csv')
ad.obs.to_csv('../scDNAm/data/GSE121364/For_Orr/Observations_table.csv')

In [221]:
ad.obs

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,DATASTORE filetype,...,Sample Type,Taxonomy,Channels,Platform_y,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date
Y4_TA_Hi_11,SRR8067241,RNA-Seq,150,107484600,PRJNA497128,SAMN10250014,41604327,GEO,public,"sra,fastq",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895580,Felix Krueger,"Aug 14, 2019"
Y4_TA_Hi_12,SRR8067242,RNA-Seq,150,90305850,PRJNA497128,SAMN10250013,35342837,GEO,public,"sra,fastq",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895581,Felix Krueger,"Aug 14, 2019"
Y4_TA_Hi_13,SRR8067243,RNA-Seq,150,97244550,PRJNA497128,SAMN10250012,37644383,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895582,Felix Krueger,"Aug 14, 2019"
Y4_TA_Hi_1,SRR8067250,RNA-Seq,150,39392850,PRJNA497128,SAMN10250005,15256244,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895589,Felix Krueger,"Aug 14, 2019"
Y4_TA_Hi_23,SRR8067254,RNA-Seq,150,116119050,PRJNA497128,SAMN10250001,45741310,GEO,public,"sra,fastq",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895593,Felix Krueger,"Aug 14, 2019"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O5_TA_Hi_90,SRR8067040,RNA-Seq,150,105824700,PRJNA497128,SAMN10249402,38171506,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895379,Felix Krueger,"Aug 14, 2019"
O5_TA_Hi_91,SRR8067041,RNA-Seq,150,107854050,PRJNA497128,SAMN10249401,38724795,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895380,Felix Krueger,"Aug 14, 2019"
O5_TA_Hi_92,SRR8067042,RNA-Seq,150,64730850,PRJNA497128,SAMN10249400,22913257,GEO,public,"fastq,sra",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895381,Felix Krueger,"Aug 14, 2019"
O5_TA_Hi_94,SRR8067044,RNA-Seq,150,158993700,PRJNA497128,SAMN10249398,58439965,GEO,public,"sra,fastq",...,SRA,Mus musculus,1,GPL17021,GSE121364;GSE121437,SRA Run Selector,https://www.ncbi.nlm.nih.gov/Traces/study/?acc...,SRX4895383,Felix Krueger,"Aug 14, 2019"


# GSE121436 scDNAm 

In [30]:
cwd = Path.cwd()
# cwd = Path('/Users/tarkhov/Dropbox/Harvard/Science/Church/')
project_dir = Path.joinpath(cwd, Path('data/GSE121436/'))
project_id = 'GSE121436'
project_samples_dir = 'GSE121436_RAW'
metainfo_path = 'SraRunTable.txt'
metainfo_path_2 = 'sample.csv'
gl_metainfo_path = Path.joinpath(project_dir, metainfo_path)
gl_metainfo_path_2 = Path.joinpath(project_dir, metainfo_path_2)

In [31]:
df_meta_2 = pd.read_csv(gl_metainfo_path_2, sep=',')
df_meta_2['CellID'] = df_meta_2['Title'].apply(lambda x: x.split(' ')[0])
print(df_meta_2.shape)
df_meta_2.head(2)

(275, 13)


,Accession,Title,Sample Type,Taxonomy,Channels,Platform,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date,CellID
0,GSM3436261,O1_TA_Hi_10 [scBS-seq],SRA,Mus musculus,1,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901189,Felix Krueger,"Aug 22, 2019",O1_TA_Hi_10
1,GSM3436262,O1_TA_Hi_11 [scBS-seq],SRA,Mus musculus,1,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901190,Felix Krueger,"Aug 22, 2019",O1_TA_Hi_11


In [32]:
df_meta = pd.read_csv(gl_metainfo_path, sep=',')
# df_meta[df_meta['Age'] == '2 months']['Mouse_ID'].unique()
df_meta[df_meta['Age'] == '24 months']['Mouse_ID'].unique()#head()

array(['O1', 'O8'], dtype=object)

In [33]:
df_meta = pd.read_csv(gl_metainfo_path, sep=',')
print(df_meta.shape)
df_meta = pd.merge(df_meta, df_meta_2, left_on='Sample Name', right_on='Accession')
print(df_meta.shape)
df_meta['Age_months'] = df_meta['Age'].copy()
df_meta['Age'] = df_meta['Age_months'].apply(lambda x: float(x.split(' ')[0]))
print(df_meta.shape)
df_meta_2 = df_meta.copy()
df_meta['Name PerCov'] = df_meta['Sample Name'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta['Data Type'] = 'percentage'
df_meta_2['Name PerCov'] = df_meta_2['Sample Name'].str.cat(['coverage'] * df_meta.shape[0], sep='_')
df_meta_2['Data Type'] = 'coverage'
df_meta['Name PerCov'] = df_meta['Sample Name'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta = pd.concat((df_meta, df_meta_2), axis=0)
df_meta.set_index('Name PerCov', inplace=True)
print(df_meta.shape)
df_meta.head()

(275, 28)
(275, 41)
(275, 42)
(550, 43)


,Run,Age,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Cell_type,Center Name,...,Platform_y,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date,CellID,Age_months,Data Type
Name PerCov,,,,,,,,,,,,,,,,,,,,,
GSM3436261_percentage,SRR8073317,24.0,Bisulfite-Seq,250,891398250,PRJNA497312,SAMN10254811,351440127,Muscle stem cells,GEO,...,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901189,Felix Krueger,"Aug 22, 2019",O1_TA_Hi_10,24 months,percentage
GSM3436262_percentage,SRR8073318,24.0,Bisulfite-Seq,250,775564750,PRJNA497312,SAMN10254810,307776943,Muscle stem cells,GEO,...,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901190,Felix Krueger,"Aug 22, 2019",O1_TA_Hi_11,24 months,percentage
GSM3436263_percentage,SRR8073319,24.0,Bisulfite-Seq,250,844106750,PRJNA497312,SAMN10254809,335133628,Muscle stem cells,GEO,...,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901191,Felix Krueger,"Aug 22, 2019",O1_TA_Hi_12,24 months,percentage
GSM3436264_percentage,SRR8073320,24.0,Bisulfite-Seq,250,947647750,PRJNA497312,SAMN10254808,376647456,Muscle stem cells,GEO,...,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901192,Felix Krueger,"Aug 22, 2019",O1_TA_Hi_13,24 months,percentage
GSM3436265_percentage,SRR8073321,24.0,Bisulfite-Seq,250,917775500,PRJNA497312,SAMN10254807,368154088,Muscle stem cells,GEO,...,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901193,Felix Krueger,"Aug 22, 2019",O1_TA_Hi_14,24 months,percentage


In [34]:
# np.intersect1d(df_meta_full.CellID.values, df_meta_2.CellID.values).shape, 
# np.intersect1d(sampleIDs, df_meta_2.CellID.values).shape

In [35]:
df_meta[df_meta['Sample Name'] == 'GSM3436480']#.iloc[275:280,:]

,Run,Age,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Cell_type,Center Name,...,Platform_y,Series,Supplementary Types,Supplementary Links,SRA Accession,Contact,Release Date,CellID,Age_months,Data Type
Name PerCov,,,,,,,,,,,,,,,,,,,,,
GSM3436480_percentage,SRR8073536,2.0,Bisulfite-Seq,252,933869412,PRJNA497312,SAMN10255048,440589845,Muscle stem cells,GEO,...,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901408,Felix Krueger,"Aug 22, 2019",Y8_TA_Hi_47,2 months,percentage
GSM3436480_coverage,SRR8073536,2.0,Bisulfite-Seq,252,933869412,PRJNA497312,SAMN10255048,440589845,Muscle stem cells,GEO,...,GPL17021,GSE121436;GSE121437,COV;SRA Run Selector,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3436...,SRX4901408,Felix Krueger,"Aug 22, 2019",Y8_TA_Hi_47,2 months,coverage


In [36]:
data_path = Path.joinpath(cwd, project_dir, "{}_coverage_0_binned_by_1.h5ad".format(project_id))
print(data_path)
ad = sc.read_h5ad(data_path)

/Users/tarkhov/Dropbox/Harvard/Science/scDNAm/data/GSE121436/GSE121436_coverage_0_binned_by_1.h5ad


In [37]:
ad

AnnData object with n_obs × n_vars = 35584147 × 550

In [38]:
dump_key="thresh_5_common_cpgs"
data_path = Path.joinpath(cwd, project_dir, f"GSE121436_wmeta_mm10{dump_key}.h5ad")
print(data_path)
ad = sc.read_h5ad(data_path)

/Users/tarkhov/Dropbox/Harvard/Science/scDNAm/data/GSE121436/GSE121436_wmeta_mm10thresh_5_common_cpgs.h5ad


In [9]:
ad

AnnData object with n_obs × n_vars = 550 × 5999943
    obs: 'Run', 'Age', 'Assay Type', 'AvgSpotLen', 'Bases', 'BioProject', 'BioSample', 'Bytes', 'Cell_type', 'Center Name', 'Consent', 'DATASTORE filetype', 'DATASTORE provider', 'DATASTORE region', 'Experiment', 'GEO_Accession (exp)', 'Instrument', 'LibraryLayout', 'LibrarySelection', 'LibrarySource', 'Mouse_ID', 'Organism', 'Platform', 'ReleaseDate', 'Sample Name', 'source_name', 'SRA Study', 'strain', 'Age_months', 'Data Type', '_full_binary'
    var: '_common_cpgs_binary', '_common_cpgs_order', '_cov>30cells_binary', '_cov>30cells_order', 'r_scMiceMuscle_full_cov>30cells_meth', 'p_scMiceMuscle_full_cov>30cells_meth', 'b_scMiceMuscle_full_cov>30cells_meth_non_sign'

In [11]:
ad.obs

,Run,Age,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Cell_type,Center Name,...,Organism,Platform,ReleaseDate,Sample Name,source_name,SRA Study,strain,Age_months,Data Type,_full_binary
GSM3436480_percentage,SRR8073536,2.0,Bisulfite-Seq,252,933869412,PRJNA497312,SAMN10255048,440589845,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436480,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,percentage,True
GSM3436480_coverage,SRR8073536,2.0,Bisulfite-Seq,252,933869412,PRJNA497312,SAMN10255048,440589845,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436480,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,coverage,True
GSM3436314_percentage,SRR8073370,24.0,Bisulfite-Seq,252,612125640,PRJNA497312,SAMN10254952,257904041,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436314,Tibialis anterior muscles,SRP166031,MGI: 5308730,24 months,percentage,True
GSM3436314_coverage,SRR8073370,24.0,Bisulfite-Seq,252,612125640,PRJNA497312,SAMN10254952,257904041,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436314,Tibialis anterior muscles,SRP166031,MGI: 5308730,24 months,coverage,True
GSM3436450_percentage,SRR8073506,2.0,Bisulfite-Seq,252,742126140,PRJNA497312,SAMN10254990,345931510,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436450,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,percentage,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3436531_coverage,SRR8073587,2.0,Bisulfite-Seq,252,1207480428,PRJNA497312,SAMN10255062,589667584,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436531,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,coverage,True
GSM3436268_percentage,SRR8073324,24.0,Bisulfite-Seq,250,1012848250,PRJNA497312,SAMN10254804,405102838,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436268,Tibialis anterior muscles,SRP166031,MGI: 5308730,24 months,percentage,True
GSM3436268_coverage,SRR8073324,24.0,Bisulfite-Seq,250,1012848250,PRJNA497312,SAMN10254804,405102838,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436268,Tibialis anterior muscles,SRP166031,MGI: 5308730,24 months,coverage,True
GSM3436365_percentage,SRR8073421,2.0,Bisulfite-Seq,250,1348635500,PRJNA497312,SAMN10255146,566490196,Muscle stem cells,GEO,...,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436365,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,percentage,True


In [13]:
ad.obs = pd.concat((ad.obs, df_meta.loc[ad.obs.index,:]), axis=1)
ad.X = ad.X.astype(np.float32)

In [215]:
# ad.obs[['Age', 'Mouse_ID']]
ad.var

,chr,start
geneName,,
Xkr4,1,3205901
Gm1992,1,3466587
Rp1,1,4343507
Sox17,1,4490928
Mrpl15,1,4773206
...,...,...
AC151712.3,Y,90754513
AC151712.1,Y,90755657
AC151712.4,Y,90784738


In [15]:
dump_key="thresh_5_common_cpgs"
data_path = Path.joinpath(cwd, project_dir, f"GSE121436_wmeta_mm10_with_CellID_{dump_key}.h5ad")

In [16]:
ad.obs_names_make_unique()
ad.var_names_make_unique()
ad.write_h5ad(data_path)

/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Run' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Age' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be 

ValueError: Unable to create dataset (name already exists)

Above error raised while writing key 'Run' of <class 'h5py._hl.group.Group'> from /.

Above error raised while writing key 'obs' of <class 'h5py._hl.files.File'> from /.

In [46]:
# import meth.tools as mtools
# df = ad.obs.copy()
# df.reset_index(inplace=True)
# df['index'] = df['index'].apply(lambda x: mtools.convert_between_assemblies(x))
# idx = df['index'].values != 'None'
# print(np.sum(idx), idx.shape[0])
# ad = ad[idx,:].copy()
# df = df[df['index'] != 'None']
# df.set_index('index', inplace=True)    
# ad.obs = df
# df.head()

In [47]:
ad.obs

""
index
chr10_100000122
chr10_100000144
chr10_100000272
chr10_100000277
chr10_100000455
...
chrY_999736
chrY_999764
chrY_9998876


In [48]:
ad.var = pd.concat((ad.var, df_meta.loc[ad.var.index,:]), axis=1)
ad.X = ad.X.astype(np.float32)

In [49]:
x = ad.X[:,::2]
xcov = ad.X[:,1::2]
xmeth = x.multiply(xcov)

mouse_ids = ad.var.loc[::2,:].Mouse_ID.values
mice = np.unique(mouse_ids)
print(mice)
xbulk = np.zeros((ad.X.shape[0], 2* len(mice)))
#xcovbulk = np.zeros((ad.X.shape[0], len(mice)))
for i,mouse_id in enumerate(mice):
    idx = mouse_ids == mouse_id
    xbulk[:,2*i] = xmeth[:,idx].sum(axis=1).flatten() / xcov[:,idx].sum(axis=1).flatten()
    xbulk[:,2*i+1] = xcov[:,idx].sum(axis=1).flatten()

['O1' 'O5' 'O8' 'Y2' 'Y7' 'Y8']


/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_16405/3837489246.py:12: RuntimeWarning: invalid value encountered in true_divide
  xbulk[:,2*i] = xmeth[:,idx].sum(axis=1).flatten() / xcov[:,idx].sum(axis=1).flatten()


In [50]:
mice_index = np.array([[f"{x}_percentage", f"{x}_coverage"] for x in mice]).flatten()

array([[0. , 1. , nan, ..., 0. , nan, 0. ],
       [1. , 1. , nan, ..., 0. , nan, 0. ],
       [0.5, 2. , nan, ..., 2. , 1. , 1. ],
       ...,
       [0. , 2. , nan, ..., 0. , nan, 0. ],
       [1. , 1. , nan, ..., 4. , nan, 0. ],
       [nan, 0. , nan, ..., 0. , nan, 0. ]])

In [52]:
df_meta_bulk = df_meta.drop_duplicates('Mouse_ID').set_index('Mouse_ID').loc[mice].copy()
df_meta_bulk.reset_index(inplace=True)
df_meta_2 = df_meta_bulk.copy()
df_meta_bulk['Name PerCov'] = df_meta_bulk['Mouse_ID'].str.cat(['percentage'] * df_meta_bulk.shape[0], sep='_')
df_meta_bulk['Data Type'] = 'percentage'
df_meta_2['Name PerCov'] = df_meta_2['Mouse_ID'].str.cat(['coverage'] * df_meta_bulk.shape[0], sep='_')
df_meta_2['Data Type'] = 'coverage'
#df_meta['Name PerCov'] = df_meta['Sample Name'].str.cat(['percentage'] * df_meta.shape[0], sep='_')
df_meta_bulk = pd.concat((df_meta_bulk, df_meta_2), axis=0)
df_meta_bulk.set_index('Name PerCov', inplace=True)
print(df_meta_bulk.shape)
df_meta_bulk = df_meta_bulk.loc[mice_index]
df_meta_bulk

(12, 30)


,Mouse_ID,Run,Age,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Cell_type,...,LibrarySource,Organism,Platform,ReleaseDate,Sample Name,source_name,SRA Study,strain,Age_months,Data Type
Name PerCov,,,,,,,,,,,,,,,,,,,,,
O1_percentage,O1,SRR8073317,24.0,Bisulfite-Seq,250,891398250,PRJNA497312,SAMN10254811,351440127,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436261,Tibialis anterior muscles,SRP166031,MGI: 5308730,24 months,percentage
O1_coverage,O1,SRR8073317,24.0,Bisulfite-Seq,250,891398250,PRJNA497312,SAMN10254811,351440127,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436261,Tibialis anterior muscles,SRP166031,MGI: 5308730,24 months,coverage
O5_percentage,O5,SRR8073544,2.0,Bisulfite-Seq,252,1151661168,PRJNA497312,SAMN10255099,568791665,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436488,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,percentage
O5_coverage,O5,SRR8073544,2.0,Bisulfite-Seq,252,1151661168,PRJNA497312,SAMN10255099,568791665,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436488,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,coverage
O8_percentage,O8,SRR8073360,24.0,Bisulfite-Seq,252,1361646216,PRJNA497312,SAMN10254897,572974528,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436304,Tibialis anterior muscles,SRP166031,MGI: 5308730,24 months,percentage
O8_coverage,O8,SRR8073360,24.0,Bisulfite-Seq,252,1361646216,PRJNA497312,SAMN10254897,572974528,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436304,Tibialis anterior muscles,SRP166031,MGI: 5308730,24 months,coverage
Y2_percentage,Y2,SRR8073408,2.0,Bisulfite-Seq,250,961632750,PRJNA497312,SAMN10255104,404404212,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436352,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,percentage
Y2_coverage,Y2,SRR8073408,2.0,Bisulfite-Seq,250,961632750,PRJNA497312,SAMN10255104,404404212,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436352,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,coverage
Y7_percentage,Y7,SRR8073448,2.0,Bisulfite-Seq,252,911255688,PRJNA497312,SAMN10254934,418481168,Muscle stem cells,...,GENOMIC,Mus musculus,ILLUMINA,2019-08-27T00:00:00Z,GSM3436392,Tibialis anterior muscles,SRP166031,MGI: 5308730,2 months,percentage


In [53]:
adbu = sc.AnnData(sp.csr_matrix(np.nan_to_num(xbulk)), ad.obs, df_meta_bulk)
adbu.obs_names_make_unique()
adbu.var_names_make_unique()

adbu.write_h5ad(Path.joinpath(project_dir, "{}_bulked_wmeta_mm10.h5ad".format(project_id)))

/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Mouse_ID' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Run' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and wil

matrix([[0. , 1. , 0. , ..., 0. , 0. , 0. ],
        [1. , 1. , 0. , ..., 0. , 0. , 0. ],
        [0.5, 2. , 0. , ..., 2. , 1. , 1. ],
        ...,
        [0. , 2. , 0. , ..., 0. , 0. , 0. ],
        [1. , 1. , 0. , ..., 4. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ]], dtype=float32)

In [55]:
ad.obs_names_make_unique()
ad.var_names_make_unique()

In [56]:
ad.write_h5ad(Path.joinpath(project_dir, "{}_wmeta.h5ad".format(project_id)))

/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Run' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Assay Type' as categorical
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and w

In [49]:
# ad.obs.reset_index(inplace=True)
# ad.obs['index'] = ad.obs['index'].apply(lambda x: "chr" + x)
# ad.obs.set_index('index', inplace=True)

# ad.var.head()
ad.X

<987360x550 sparse matrix of type '<class 'numpy.float32'>'
	with 2773548 stored elements in Compressed Sparse Row format>

In [24]:
# ad.var[ad.var['Age'] == 2.0]['Mouse_ID'].unique()
# ad.var[ad.var['Age'] == 24.0]['Mouse_ID'].unique()

In [18]:
ad.X[:100,:2]

<100x2 sparse matrix of type '<class 'numpy.float32'>'
	with 21 stored elements in Compressed Sparse Row format>

In [31]:
from pyliftover import LiftOver

lo_mm9_to_mm10 = LiftOver('meth/mm9ToMm10.over.chain.gz')
lo_mm10_to_mm39 = LiftOver('meth/mm10ToMm39.over.chain.gz')


In [38]:
lo_mm9_to_mm10.chain_file.chain_index['chr1']

In [57]:
ad.write_h5ad(Path.joinpath(project_dir, "{}_wmeta_mm10.h5ad".format(project_id)))

# GSE121436 — preprocessing raw files

In [112]:
import pandas as pd
import os
import numpy as np
import sys
import pwd
from pathlib import Path
from tqdm import tqdm, trange
from tqdm.contrib.concurrent import process_map
import meth.tools as mtools

cwd = Path.cwd()
project_dir = Path('data/GSE121436')
project_id = 'GSE121436'
project_samples_dir = 'GSE121436_RAW'
path_samples = Path.joinpath(project_dir, project_samples_dir)
gl_path_samples = Path.joinpath(cwd, path_samples)
gl_project_dir = Path.joinpath(cwd, project_dir)
all_samples = os.listdir(gl_path_samples)
print(len(all_samples))
# print(all_samples)
df_glob = None
id_to_gsm = dict(zip([x.split('_')[1] for x in all_samples], [x.split('_')[0] for x in all_samples]))

coverage = 0
bin_by = 10

sample_path_tuples = [Path.joinpath(gl_path_samples, file) for file in all_samples]
path_sample = sample_path_tuples[0]
df = pd.read_csv(path_sample, sep='\t', header=None)
gsm = path_sample.name.split('_')[0]
df[0] = df[0].astype(str)
df['index'] = df.iloc[:,0].str.cat(df.iloc[:,1].astype(str), sep="_")
df.set_index('index', inplace=True)
df['{}_percentage'.format(gsm)] = df.iloc[:,3]
df['{}_coverage'.format(gsm)] = df.iloc[:,4] + df.iloc[:,5]
df.drop(labels=np.arange(6), axis=1, inplace=True)
df = df[df.iloc[:,1] >= coverage]
df = mtools.get_binned_cpgs(df, bin_by=bin_by)
df['{}_percentage'.format(gsm)] = 1. * df['{}_percentage'.format(gsm)].values / df['{}_coverage'.format(gsm)].values
df.dropna(axis=0, how='any', inplace=True)
print(df.dtypes)
df = df.astype(np.float16)
print(df.dtypes)
df.head()

275


/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


GSM3436480_percentage    float64
GSM3436480_coverage        int64
dtype: object
GSM3436480_percentage    float16
GSM3436480_coverage      float16
dtype: object


,GSM3436480_percentage,GSM3436480_coverage
index,,
10_100001640,100.0,1.0
10_100001650,100.0,1.0
10_100001680,100.0,2.0
10_10000420,100.0,1.0
10_100004300,0.0,1.0


In [106]:
pd.__version__

'1.3.5'

In [113]:
dfs = pd.DataFrame({'{}_percentage'.format(gsm) : pd.arrays.SparseArray(df['{}_percentage'.format(gsm)].values, fill_value=np.nan),
                    '{}_coverage'.format(gsm) : pd.arrays.SparseArray(df['{}_coverage'.format(gsm)].values, fill_value=np.nan),
                   }, index=df.index)
# dfs = pd.DataFrame({'{}_percentage'.format(gsm) : pd.arrays.SparseArray(df['{}_percentage'.format(gsm)].values),
#                     '{}_coverage'.format(gsm) : pd.arrays.SparseArray(df['{}_coverage'.format(gsm)].values),
#                    }, index=df.index)

In [111]:
# dfs2 = pd.DataFrame({'{}_percentage'.format(gsm) : pd.arrays.SparseArray(df['{}_percentage'.format(gsm)].values),
#                     '{}_coverage'.format(gsm) : pd.arrays.SparseArray(df['{}_coverage'.format(gsm)].values),
#                    }, index=df.index)
dfs2 = pd.DataFrame({'{}_percentage'.format(gsm) : pd.arrays.SparseArray(df['{}_percentage'.format(gsm)].values, fill_value=np.nan),
                    '{}_coverage'.format(gsm) : pd.arrays.SparseArray(df['{}_coverage'.format(gsm)].values, fill_value=np.nan),
                   }, index=df.index)

In [98]:
# dfs
dfs2.dtypes

GSM3436314_percentage    Sparse[float64, nan]
GSM3436314_coverage          Sparse[int64, 0]
dtype: object

In [121]:
an1 = sc.AnnData(dfs)
an2 = sc.AnnData(dfs2)

In [133]:
an1, an2
anres = sc.concat((an1,an2, an3), axis=1, join='outer')
anres

AnnData object with n_obs × n_vars = 1453605 × 4

In [135]:
# anres.obs, an2.obs, an1.obs

In [114]:
dfres = pd.concat((dfs, dfs2), axis=1)
dfres2 = dfres.fillna(0)
dfres.head()

,GSM3436480_percentage,GSM3436480_coverage,GSM3436314_percentage,GSM3436314_coverage
index,,,,
10_100001640,100.0,1.0,NaN,NaN
10_100001650,100.0,1.0,NaN,NaN
10_100001680,100.0,2.0,NaN,NaN
10_10000420,100.0,1.0,NaN,NaN
10_100004300,0.0,1.0,NaN,NaN


In [100]:
dfres2.head()

,GSM3436480_percentage,GSM3436480_coverage,GSM3436314_percentage,GSM3436314_coverage
index,,,,
10_100001640,100.0,1.0,0.0,0.0
10_100001650,100.0,1.0,0.0,0.0
10_100001680,100.0,2.0,0.0,0.0
10_10000420,100.0,1.0,0.0,0.0
10_100004300,0.0,1.0,0.0,0.0


In [115]:
dfres['GSM3436314_coverage'].values

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, ...]
Fill: nan
IntIndex
Indices: array([   1382,    2945,    3130, ..., 1453602, 1453603, 1453604],
      dtype=int32)

In [116]:
dfres2['GSM3436314_coverage'].values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]
Fill: 0
IntIndex
Indices: array([   1382,    2945,    3130, ..., 1453602, 1453603, 1453604],
      dtype=int32)

In [117]:
dfres2.dtypes

GSM3436480_percentage    Sparse[float64, 0]
GSM3436480_coverage      Sparse[float64, 0]
GSM3436314_percentage    Sparse[float64, 0]
GSM3436314_coverage      Sparse[float64, 0]
dtype: object

In [118]:
dfres.dtypes

GSM3436480_percentage    Sparse[float64, nan]
GSM3436480_coverage      Sparse[float64, nan]
GSM3436314_percentage    Sparse[float64, nan]
GSM3436314_coverage      Sparse[float64, nan]
dtype: object

In [119]:
print("df sparse size =", dfs.memory_usage().sum())
print("df2 sparse size =", dfs2.memory_usage().sum())
print("dfsum sparse size =", dfres.memory_usage().sum())
print("dfsum2 sparse size =", dfres2.memory_usage().sum())


df sparse size = 62839836
df2 sparse size = 196816
dfsum sparse size = 46534320
dfsum2 sparse size = 46534320


In [120]:
print("df sparse size =", dfs.memory_usage().sum())
print("df2 sparse size =", dfs2.memory_usage().sum())
print("dfsum sparse size =", dfres.memory_usage().sum())
print("dfsum2 sparse size =", dfres2.memory_usage().sum())


df sparse size = 62839836
df2 sparse size = 196816
dfsum sparse size = 46534320
dfsum2 sparse size = 46534320


In [5]:
df = pd.DataFrame(np.arange(10), columns=['Dat'])
df.index = np.arange(10) + 100
df.head()

,Dat
100,0
101,1
102,2
103,3
104,4


In [7]:
dfs = pd.DataFrame({'Dat_sp': pd.arrays.SparseArray(df['Dat'].values)},
                  )#index=df.index)
dfs.head()

,Dat_sp
0,0
1,1
2,2
3,3
4,4
